In [1]:
import os
import time
from keras.applications.imagenet_utils import preprocess_input
import numpy as np  
np.set_printoptions(threshold=np.inf)
import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)

from keras.preprocessing.image import ImageDataGenerator
from keras import layers, activations
from keras.models import Model
from keras.layers import Input,Dense,Flatten,Dropout,ZeroPadding2D,BatchNormalization,Activation,Add,Dot,AveragePooling2D,Lambda,Reshape
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D,MaxPooling2D,DepthwiseConv2D  

Using TensorFlow backend.


In [2]:
dataset = 'UCM'
base_path='./Dataset/' + dataset
train_path=os.path.join(base_path, 'Train')
ModelName = "Gabor-ResNeXt50"

All_Labels = os.listdir(train_path)
num_classes = len(All_Labels)

In [3]:
from tensorflow.keras.callbacks import Callback
import tensorflow.keras.backend as K
class WarmupExponentialDecay(Callback):
    def __init__(self,lr_base=0.0002,lr_min=0.0,decay=0,warmup_epochs=0):
        self.num_passed_batchs = 0   #一个计数器
        self.warmup_epochs=warmup_epochs  
        self.lr=lr_base #learning_rate_base
        self.lr_min=lr_min #最小的起始学习率,此代码尚未实现
        self.decay=decay  #指数衰减率
        self.steps_per_epoch=0 #也是一个计数器
    def on_batch_begin(self, batch, logs=None):
        # params是模型自动传递给Callback的一些参数
        if self.steps_per_epoch==0:
            #防止跑验证集的时候呗更改了
            if self.params['steps'] == None:
                self.steps_per_epoch = np.ceil(1. * self.params['samples'] / self.params['batch_size'])
            else:
                self.steps_per_epoch = self.params['steps']
        if self.num_passed_batchs < self.steps_per_epoch * self.warmup_epochs:
            K.set_value(self.model.optimizer.lr,
                        self.lr*(self.num_passed_batchs + 1) / self.steps_per_epoch / self.warmup_epochs)
        else:
            K.set_value(self.model.optimizer.lr,
                        self.lr*((1-self.decay)**(self.num_passed_batchs-self.steps_per_epoch*self.warmup_epochs)))
        self.num_passed_batchs += 1
    def on_epoch_begin(self,epoch,logs=None):
    #用来输出学习率的,可以删除
        print("learning_rate:",K.get_value(self.model.optimizer.lr))  

In [4]:
import cv2
def Gabor(shape, dtype=None):    
    ksize = (7, 7)
    # 核尺寸
    #sigmas = [1] # [2, 4]
    # 角度
    #thetas = np.linspace(0, 2*np.pi, 8, endpoint=False) # np.linspace(0, np.pi, 4, endpoint=False)
    # 波长(间隔)
    #lambdas = np.linspace(2, 3, 6) # [8, 16, 32, 64]
    # 高度(越小，核函数图像会越高)
    #gammas = [1] # np.linspace(1, 0, 2, endpoint=False)
    # 中轴
    #psis = [0, 2*np.pi]
    
    gabors = []
    
    for i in range(0,int(64/4)):    
#     size, sigma, theta, lambda, gamma aspect ratio                 
        gf = cv2.getGaborKernel(ksize=ksize, sigma=1, theta=0, lambd=2, gamma=1, psi=0, ktype=cv2.CV_32F)
        gabors.append(gf)             
        gf = cv2.getGaborKernel(ksize=ksize, sigma=1, theta=np.pi/2, lambd=2, gamma=1, psi=0, ktype=cv2.CV_32F)
        gabors.append(gf)             
        gf = cv2.getGaborKernel(ksize=ksize, sigma=1, theta=np.pi/4, lambd=2, gamma=1, psi=0, ktype=cv2.CV_32F)
        gabors.append(gf)             
        gf = cv2.getGaborKernel(ksize=ksize, sigma=1, theta=np.pi/4*3, lambd=2, gamma=1, psi=0, ktype=cv2.CV_32F)
        gabors.append(gf)
    stacked_list = np.array([gabors])
    stacked_list = np.einsum('hijk->jkhi', stacked_list)
    
    b = K.constant(stacked_list, dtype='float32')
    F_0 = Lambda(lambda x: K.cast(x, dtype='float32'))(b)
    return F_0

In [5]:
def block3(x, filters, kernel_size=3, stride=1, groups=32, conv_shortcut=True, name=None):
    bn_axis = 3 if tf.keras.backend.image_data_format() == 'channels_last' else 1

    if conv_shortcut is True:
        shortcut = Conv2D((64 // groups) * filters, 1, strides=stride, use_bias=False, name=name + '_0_conv')(x)
        shortcut = BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_0_bn')(shortcut)
    else:
        shortcut = x

    x = Conv2D(filters, 1, use_bias=False, name=name + '_1_conv')(x)
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_1_bn')(x)
    x = Activation('relu', name=name + '_1_relu')(x)

    c = filters // groups
    x = ZeroPadding2D(padding=((1, 1), (1, 1)), name=name + '_2_pad')(x)
    x = DepthwiseConv2D(kernel_size, strides=stride, depth_multiplier=c, use_bias=False, name=name + '_2_conv')(x)
    x_shape = tf.keras.backend.int_shape(x)[1:-1]
    x = Reshape(x_shape + (groups, c, c))(x)
    output_shape = x_shape + (groups, c) if tf.keras.backend.backend() == 'theano' else None
    x = Lambda(lambda x: sum([x[:, :, :, :, i] for i in range(c)]),
                      output_shape=output_shape, name=name + '_2_reduce')(x)
    x = Reshape(x_shape + (filters,))(x)
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_2_bn')(x)
    x = Activation('relu', name=name + '_2_relu')(x)

    x = Conv2D((64 // groups) * filters, 1, use_bias=False, name=name + '_3_conv')(x)
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_3_bn')(x)

    x = Add(name=name + '_add')([shortcut, x])
    x = Activation('relu', name=name + '_out')(x)
    return x

def stack3(x, filters, blocks, stride1=2, groups=32, name=None):
    x = block3(x, filters, stride=stride1, groups=groups, name=name + '_block1')
    for i in range(2, blocks + 1):
        x = block3(x, filters, groups=groups, conv_shortcut=False, name=name + '_block' + str(i))
    return x

def stack_fn(x):
    x = stack3(x, 128, 3, stride1=1, name='conv2')
    x = stack3(x, 256, 4, name='conv3')
    x = stack3(x, 512, 6, name='conv4')
    x = stack3(x, 1024, 3, name='conv5')
    return x

from keras.optimizers import SGD  
def Gabor_ResNeXt50(shape):
    if tf.keras.backend.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
        
    img_input = Input(shape=shape)  

    bn_axis = 3 if tf.keras.backend.image_data_format() == 'channels_last' else 1

    x = ZeroPadding2D(padding=((3, 3), (3, 3)), name='conv1_pad')(img_input)
    x = Conv2D(64, 7, strides=2, use_bias=False, name='conv1_conv')(x)
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name='conv1_bn')(x)
    x = Activation('relu', name='conv1_relu')(x)
    x = ZeroPadding2D(padding=((3, 3), (3, 3)))(x)
    x = Conv2D(64, (7, 7), strides=1, use_bias=False, kernel_initializer=Gabor, name='Gabor')(x)
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name='Gabor-Batch')(x)
    x = Activation('relu', name='Gabor-relu')(x)
    
    x = ZeroPadding2D(padding=((1, 1), (1, 1)), name='pool1_pad')(x)
    x = MaxPooling2D(3, strides=2, name='pool1_pool')(x)
    x = stack_fn(x)
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name='post_bn')(x)
    x = Activation('relu', name='post_relu')(x)
    x = GlobalAveragePooling2D(name='avg_pool')(x)
    x = Dense(1024, activation='relu',name='fc1024')(x)
    x = Dense(512, activation='relu',name='fc512')(x)
    x = Dense(num_classes, activation='softmax', name='probs')(x)


    model = Model(inputs=img_input,outputs=x, name=ModelName)  
    sgd = SGD(decay=0.0001,momentum=0.9)  
    model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])  

    return model

In [6]:
K.clear_session()
tf.reset_default_graph()

In [7]:
model = Gabor_ResNeXt50([224,224,3])
model.load_weights('./models/ResNeXt50.h5', by_name=True)
#model.load_weights('./models/2021-02-20/UCM/Gabor-ResNeXt50.h5', by_name=True)
model.summary()
#model.layers[6].trainable = False

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "Gabor-ResNeXt50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9408        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
__

In [8]:
batch_size=20
epochs = 800
data_gen = ImageDataGenerator(horizontal_flip=True,
                              vertical_flip=True,
                              preprocessing_function = preprocess_input,
                              rotation_range = 90)
train_generator = data_gen.flow_from_directory(train_path,
                                               target_size=(224, 224),
                                               batch_size=batch_size,
                                               class_mode='categorical',
                                               shuffle=True)
history = model.fit_generator(generator=train_generator, epochs=epochs,
                              callbacks=[WarmupExponentialDecay(lr_base=0.0002,decay=0.00002,warmup_epochs=2)])

Found 1680 images belonging to 21 classes.

Epoch 1/800
learning_rate: 0.01
84/84 [==============================] - 51s 608ms/step - loss: 3.1055 - accuracy: 0.0512
Epoch 2/800
learning_rate: 1e-04
84/84 [==============================] - 42s 505ms/step - loss: 2.7591 - accuracy: 0.2452
Epoch 3/800
learning_rate: 0.0002
84/84 [==============================] - 43s 513ms/step - loss: 2.0670 - accuracy: 0.5815
Epoch 4/800
learning_rate: 0.00019966827
84/84 [==============================] - 43s 514ms/step - loss: 1.3418 - accuracy: 0.7661
Epoch 5/800
learning_rate: 0.00019933311
84/84 [==============================] - 43s 516ms/step - loss: 0.8540 - accuracy: 0.8458
Epoch 6/800
learning_rate: 0.0001989985
84/84 [==============================] - 43s 517ms/step - loss: 0.5348 - accuracy: 0.9018
Epoch 7/800
learning_rate: 0.00019866446
84/84 [==============================] - 44s 519ms/step - loss: 0.3856 - accuracy: 0.9083
Epoch 8/800
learning_rate: 0.00019833099
84/84 [================

84/84 [==============================] - 44s 528ms/step - loss: 0.0184 - accuracy: 0.9964
Epoch 62/800
learning_rate: 0.00018113025
84/84 [==============================] - 44s 527ms/step - loss: 0.0217 - accuracy: 0.9958
Epoch 63/800
learning_rate: 0.0001808262
84/84 [==============================] - 44s 527ms/step - loss: 0.0198 - accuracy: 0.9958
Epoch 64/800
learning_rate: 0.00018052266
84/84 [==============================] - 44s 527ms/step - loss: 0.0138 - accuracy: 0.9982
Epoch 65/800
learning_rate: 0.00018021963
84/84 [==============================] - 44s 529ms/step - loss: 0.0149 - accuracy: 0.9976
Epoch 66/800
learning_rate: 0.00017991712
84/84 [==============================] - 44s 528ms/step - loss: 0.0205 - accuracy: 0.9946
Epoch 67/800
learning_rate: 0.0001796151
84/84 [==============================] - 44s 528ms/step - loss: 0.0194 - accuracy: 0.9958
Epoch 68/800
learning_rate: 0.0001793136
84/84 [==============================] - 44s 529ms/step - loss: 0.0190 - accura

84/84 [==============================] - 44s 530ms/step - loss: 0.0097 - accuracy: 0.9982
Epoch 124/800
learning_rate: 0.00016321287
84/84 [==============================] - 45s 530ms/step - loss: 0.0109 - accuracy: 0.9970
Epoch 125/800
learning_rate: 0.0001629389
84/84 [==============================] - 45s 530ms/step - loss: 0.0079 - accuracy: 0.9994
Epoch 126/800
learning_rate: 0.0001626654
84/84 [==============================] - 45s 531ms/step - loss: 0.0092 - accuracy: 0.9982
Epoch 127/800
learning_rate: 0.00016239235
84/84 [==============================] - 45s 530ms/step - loss: 0.0090 - accuracy: 0.9988
Epoch 128/800
learning_rate: 0.00016211974
84/84 [==============================] - 44s 529ms/step - loss: 0.0109 - accuracy: 0.9982
Epoch 129/800
learning_rate: 0.00016184761
84/84 [==============================] - 44s 529ms/step - loss: 0.0098 - accuracy: 0.9994
Epoch 130/800
learning_rate: 0.00016157594
84/84 [==============================] - 45s 531ms/step - loss: 0.0094 

84/84 [==============================] - 45s 534ms/step - loss: 0.0059 - accuracy: 0.9994
Epoch 185/800
learning_rate: 0.00014731516
84/84 [==============================] - 45s 535ms/step - loss: 0.0063 - accuracy: 0.9994
Epoch 186/800
learning_rate: 0.00014706788
84/84 [==============================] - 45s 534ms/step - loss: 0.0065 - accuracy: 0.9982
Epoch 187/800
learning_rate: 0.00014682101
84/84 [==============================] - 45s 534ms/step - loss: 0.0072 - accuracy: 0.9994
Epoch 188/800
learning_rate: 0.00014657456
84/84 [==============================] - 45s 534ms/step - loss: 0.0055 - accuracy: 0.9988
Epoch 189/800
learning_rate: 0.00014632852
84/84 [==============================] - 45s 534ms/step - loss: 0.0047 - accuracy: 1.0000
Epoch 190/800
learning_rate: 0.0001460829
84/84 [==============================] - 45s 535ms/step - loss: 0.0048 - accuracy: 0.9994
Epoch 191/800
learning_rate: 0.00014583768
84/84 [==============================] - 45s 535ms/step - loss: 0.0056

84/84 [==============================] - 45s 540ms/step - loss: 0.0065 - accuracy: 0.9988
Epoch 247/800
learning_rate: 0.00013274277
84/84 [==============================] - 45s 540ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 248/800
learning_rate: 0.00013251996
84/84 [==============================] - 45s 541ms/step - loss: 0.0035 - accuracy: 0.9994
Epoch 249/800
learning_rate: 0.00013229751
84/84 [==============================] - 45s 540ms/step - loss: 0.0050 - accuracy: 0.9994
Epoch 250/800
learning_rate: 0.00013207544
84/84 [==============================] - 45s 541ms/step - loss: 0.0061 - accuracy: 0.9982
Epoch 251/800
learning_rate: 0.00013185373
84/84 [==============================] - 45s 541ms/step - loss: 0.0055 - accuracy: 0.9994
Epoch 252/800
learning_rate: 0.0001316324
84/84 [==============================] - 45s 540ms/step - loss: 0.0047 - accuracy: 1.0000
Epoch 253/800
learning_rate: 0.00013141143
84/84 [==============================] - 45s 540ms/step - loss: 0.0060

...

84/84 [==============================] - 45s 533ms/step - loss: 0.0042 - accuracy: 0.9988
Epoch 492/800
learning_rate: 8.795358e-05
84/84 [==============================] - 45s 533ms/step - loss: 0.0043 - accuracy: 0.9988
Epoch 493/800
learning_rate: 8.7805936e-05
84/84 [==============================] - 45s 532ms/step - loss: 0.0027 - accuracy: 1.0000
Epoch 494/800
learning_rate: 8.765855e-05
84/84 [==============================] - 45s 531ms/step - loss: 0.0044 - accuracy: 0.9994
Epoch 495/800
learning_rate: 8.7511406e-05
84/84 [==============================] - 45s 532ms/step - loss: 0.0034 - accuracy: 0.9994
Epoch 496/800
learning_rate: 8.7364504e-05
84/84 [==============================] - 45s 532ms/step - loss: 0.0040 - accuracy: 1.0000
Epoch 497/800
learning_rate: 8.721786e-05
84/84 [==============================] - 45s 531ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 498/800
learning_rate: 8.707145e-05
84/84 [==============================] - 45s 532ms/step - loss: 0.0059 - 

84/84 [==============================] - 45s 532ms/step - loss: 0.0054 - accuracy: 0.9994
Epoch 554/800
learning_rate: 7.925322e-05
84/84 [==============================] - 45s 530ms/step - loss: 0.0057 - accuracy: 0.9982
Epoch 555/800
learning_rate: 7.912019e-05
84/84 [==============================] - 45s 532ms/step - loss: 0.0083 - accuracy: 0.9982
Epoch 556/800
learning_rate: 7.898738e-05
84/84 [==============================] - 45s 532ms/step - loss: 0.0028 - accuracy: 0.9994
Epoch 557/800
learning_rate: 7.885479e-05
84/84 [==============================] - 45s 531ms/step - loss: 0.0027 - accuracy: 0.9994
Epoch 558/800
learning_rate: 7.872243e-05
84/84 [==============================] - 45s 531ms/step - loss: 0.0028 - accuracy: 0.9994
Epoch 559/800
learning_rate: 7.859028e-05
84/84 [==============================] - 45s 532ms/step - loss: 0.0038 - accuracy: 0.9988
Epoch 560/800
learning_rate: 7.845836e-05
84/84 [==============================] - 45s 532ms/step - loss: 0.0044 - acc

84/84 [==============================] - 45s 531ms/step - loss: 0.0037 - accuracy: 0.9994
Epoch 616/800
learning_rate: 7.141351e-05
84/84 [==============================] - 45s 531ms/step - loss: 0.0028 - accuracy: 0.9994
Epoch 617/800
learning_rate: 7.129364e-05
84/84 [==============================] - 45s 531ms/step - loss: 0.0060 - accuracy: 0.9994
Epoch 618/800
learning_rate: 7.117396e-05
84/84 [==============================] - 45s 531ms/step - loss: 0.0045 - accuracy: 0.9988
Epoch 619/800
learning_rate: 7.1054485e-05
84/84 [==============================] - 45s 531ms/step - loss: 0.0095 - accuracy: 0.9976
Epoch 620/800
learning_rate: 7.093522e-05
84/84 [==============================] - 45s 531ms/step - loss: 0.0045 - accuracy: 0.9982
Epoch 621/800
learning_rate: 7.0816146e-05
84/84 [==============================] - 45s 533ms/step - loss: 0.0039 - accuracy: 0.9988
Epoch 622/800
learning_rate: 7.069727e-05
84/84 [==============================] - 45s 532ms/step - loss: 0.0049 - a

84/84 [==============================] - 45s 533ms/step - loss: 0.0031 - accuracy: 1.0000
Epoch 678/800
learning_rate: 6.43493e-05
84/84 [==============================] - 45s 531ms/step - loss: 0.0066 - accuracy: 0.9988
Epoch 679/800
learning_rate: 6.424128e-05
84/84 [==============================] - 45s 532ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 680/800
learning_rate: 6.413345e-05
84/84 [==============================] - 45s 531ms/step - loss: 0.0025 - accuracy: 0.9994
Epoch 681/800
learning_rate: 6.402579e-05
84/84 [==============================] - 45s 532ms/step - loss: 0.0035 - accuracy: 0.9994
Epoch 682/800
learning_rate: 6.391831e-05
84/84 [==============================] - 45s 540ms/step - loss: 0.0032 - accuracy: 0.9994
Epoch 683/800
learning_rate: 6.381102e-05
84/84 [==============================] - 46s 542ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 684/800
learning_rate: 6.370391e-05
84/84 [==============================] - 46s 542ms/step - loss: 0.0021 - accu

84/84 [==============================] - 45s 533ms/step - loss: 0.0060 - accuracy: 0.9988
Epoch 740/800
learning_rate: 5.7983878e-05
84/84 [==============================] - 45s 532ms/step - loss: 0.0050 - accuracy: 0.9994
Epoch 741/800
learning_rate: 5.7886544e-05
84/84 [==============================] - 45s 532ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 742/800
learning_rate: 5.7789377e-05
84/84 [==============================] - 45s 531ms/step - loss: 0.0040 - accuracy: 1.0000
Epoch 743/800
learning_rate: 5.769237e-05
84/84 [==============================] - 45s 532ms/step - loss: 0.0042 - accuracy: 0.9988
Epoch 744/800
learning_rate: 5.7595527e-05
84/84 [==============================] - 45s 531ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 745/800
learning_rate: 5.7498848e-05
84/84 [==============================] - 45s 533ms/step - loss: 0.0035 - accuracy: 0.9994
Epoch 746/800
learning_rate: 5.740233e-05
84/84 [==============================] - 45s 533ms/step - loss: 0.0029 

In [9]:
import time
save_folder = 'models/' + time.strftime("%Y-%m-%d", time.localtime()) + '/' + dataset
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

# serialize model to JSON
#import pickle
model_json = model.to_json()
with open(os.path.join(save_folder, ModelName + ".json"), "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save(os.path.join(save_folder, ModelName + ".h5"))
#pickle.dump(history.history, open('history/UCMerced_LandUse/AlexNet.p','wb'))
print("Saved Model to disk")

Saved Model to disk


In [10]:
def read_image(image_name):
    im=cv2.imread(image_name)
    im=cv2.resize(im, (224, 224))
    return im

#数据扩充
def img_Rotation(img,angel):
    rows,cols=img.shape[:2]
    #90度旋转
    M=cv2.getRotationMatrix2D((cols/2,rows/2),angel,1)
    dst=cv2.warpAffine(img,M,(cols,rows))
    
    return dst

In [11]:
Extension='.tif'
All_Labels = os.listdir(os.path.join(base_path, 'Test'))
images = []
labels = []
outputFileName = []

save_folder = 'FeatureMap/' + time.strftime("%Y-%m-%d", time.localtime()) + '/' + ModelName + '/' + dataset  +'/'

for i in range(0, num_classes):
    file_dir = os.path.join(base_path, 'Test', All_Labels[i])
    file_names = os.listdir(file_dir)
    for file_name in file_names:
        if(file_name.endswith(Extension)):
            finalFileName = os.path.join(file_dir, file_name)
            Label = np.linspace(0, 0, num_classes, dtype='int32')
            Label[i] = 1    
            images.append(read_image(finalFileName))
            labels.append(Label)
            outputFileName.append(os.path.join(save_folder, All_Labels[i], file_name).replace(Extension, ".txt"))

print("Get %d Test Images" %(len(images)))
output = np.array(images, dtype="float32")
output = preprocess_input(output)

Get 420 Test Images


In [12]:
import os
import time
from keras.models import Model  
OutPutLayer = Model(inputs=model.input, outputs=model.get_layer('fc512').output)
print("Saved FeatureMap to disk...")
OutputFeatures=[]
for i in range(0, len(output)):
    p = OutPutLayer.predict(output[i : i + 1])
    out=np.reshape(p,p.shape[1])
    OutputFeatures.append(out)
    print("\r当前输出：%d" %(i + 1), end= " ")

OutputFeatures = np.array(OutputFeatures, dtype="float") 
save_folder = 'FeatureMap/' + time.strftime("%Y-%m-%d", time.localtime()) + '/' + ModelName + '/' + dataset  +'/'
if not os.path.exists(save_folder):
    os.makedirs(save_folder)
np.savetxt(os.path.join(save_folder, dataset + '-Test-2048D.txt'), OutputFeatures)
print("\n保存完成！")
print(OutputFeatures.shape)

Saved FeatureMap to disk...
当前输出：420  
保存完成！
(420, 512)


In [13]:
Extension='.tif'
All_Labels = os.listdir(os.path.join(base_path, 'Train'))
images = []
labels = []
outputFileName = []

save_folder = 'FeatureMap/' + time.strftime("%Y-%m-%d", time.localtime()) + '/' + ModelName + '/' + dataset  +'/'

for i in range(0, num_classes):
    file_dir = os.path.join(base_path, 'Train', All_Labels[i])
    file_names = os.listdir(file_dir)
    for file_name in file_names:
        if(file_name.endswith(Extension)):
            finalFileName = os.path.join(file_dir, file_name)
            Label = np.linspace(0, 0, num_classes, dtype='int32')
            Label[i] = 1    
            images.append(read_image(finalFileName))
            labels.append(Label)
            outputFileName.append(os.path.join(save_folder, All_Labels[i], file_name).replace(Extension, ".txt"))

print("Get %d Test Images" %(len(images)))
output = np.array(images, dtype="float32")
output = preprocess_input(output)

import os
import time
from keras.models import Model  
OutPutLayer = Model(inputs=model.input, outputs=model.get_layer('fc512').output)
print("Saved FeatureMap to disk...")
OutputFeatures=[]
for i in range(0, len(output)):
    p = OutPutLayer.predict(output[i : i + 1])
    out=np.reshape(p,p.shape[1])
    OutputFeatures.append(out)
    print("\r当前输出：%d" %(i + 1), end= " ")

OutputFeatures = np.array(OutputFeatures, dtype="float") 
save_folder = 'FeatureMap/' + time.strftime("%Y-%m-%d", time.localtime()) + '/' + ModelName + '/' + dataset  +'/'
if not os.path.exists(save_folder):
    os.makedirs(save_folder)
np.savetxt(os.path.join(save_folder, dataset + '-Train-2048D.txt'), OutputFeatures)
print("\n保存完成！")
print(OutputFeatures.shape)

Get 1680 Test Images
Saved FeatureMap to disk...
当前输出：1680      
保存完成！
(1680, 512)
